In [160]:
import os
import re
import pandas as pd

In [161]:
folder_path = 'data_raw'
json_files = [f for f in os.listdir(folder_path) if f.endswith('.json')]
dfs = []
for file in json_files:
    file_path = os.path.join(folder_path, file)
    data = pd.read_json(file_path)
    dfs.append(data)
df = pd.concat(dfs, ignore_index=True)

In [162]:
def denumber(x):
    if isinstance(x, str):
        while x[-1] in ['0','1','2','3','4','5','6','7','8','9']:
            x = x[:-1]
        return x.strip()
    else:
        return x

In [163]:
df['competition'].drop_duplicates().to_list()

['1955–56 European Cup',
 '1955–58 Inter-Cities Fairs Cup',
 '1956–57 European Cup',
 '1957–58 European Cup',
 '1958–59 European Cup',
 '1958–60 Inter-Cities Fairs Cup',
 '1959–60 European Cup',
 '1960–61 European Cup',
 "1960–61 European Cup Winners' Cup",
 '1960–61 Inter-Cities Fairs Cup',
 '1961–62 European Cup',
 "1961–62 European Cup Winners' Cup",
 '1961–62 Inter-Cities Fairs Cup',
 '1962–63 European Cup',
 "1962–63 European Cup Winners' Cup",
 '1962–63 Inter-Cities Fairs Cup',
 '1963–64 European Cup',
 "1963–64 European Cup Winners' Cup",
 '1963–64 Inter-Cities Fairs Cup',
 '1964–65 European Cup',
 "1964–65 European Cup Winners' Cup",
 '1964–65 Inter-Cities Fairs Cup',
 '1965–66 European Cup',
 "1965–66 European Cup Winners' Cup",
 '1965–66 Inter-Cities Fairs Cup',
 '1966–67 European Cup',
 "1966–67 European Cup Winners' Cup",
 '1966–67 Inter-Cities Fairs Cup',
 '1967–68 European Cup',
 "1967–68 European Cup Winners' Cup",
 '1967–68 Inter-Cities Fairs Cup',
 '1968–69 European Cu

In [164]:
def tell_competition(x):
    x = str(x)
    if "Cup Winners" in x:
        return "CWC"
    elif ("European Cup" in x) and ('Winners') not in x:
        return "UCL"
    elif "Champions League" in x:
        return "UCL"
    elif ("UEFA Cup" in x) and ('Winners') not in x:
        return "EL"
    elif "Europa League" in x:
        return "EL"
    elif "Conference League" in x:
        return "UECL"
    elif "Inter-Cities Fairs Cup" in x:
        return "ICFC"
    elif "Intertoto" in x:
        return "Intertoto"
    else:
        print(x)
        return None

In [165]:
def tell_season(x):
    x = str(x)
    try:
        season = re.search('\d{4}–{0,1}\d{0,4}',x).group()
        season = season.replace('–','-')
        if len(season) == 9:
            season = season[0:5] + season[-2:]
        elif len(season) == 4:
            season = f"{int(season) - 1}-{season[-2:]}"
        return season
    except:
        return None

In [166]:
df['competitonOriginal'] = df['competition']

In [167]:
df['competition'] = df['competitonOriginal'].apply(lambda x: tell_competition(x))

nan


In [168]:
df['season'] = df['competitonOriginal'].apply(lambda x: tell_season(x))

In [169]:
df.groupby('competition').size().nlargest()

competition
EL           13440
UCL           8206
CWC           1986
Intertoto     1561
UECL          1174
dtype: int64

In [170]:
df['location'] = df['location'].apply(lambda x: denumber(x))

In [171]:
df['date'] = pd.to_datetime(df['date'], errors='coerce')

In [172]:
df = df.sort_values(by='date')

In [173]:
df

,competition,date,score,homeTeam,homeTeamCountry,homeTeamUrl,awayTeam,awayTeamCountry,awayTeamUrl,location,phase,attendance,competitonOriginal,season
1075,CWC,1932-04-14,1-0,Celtic,Scotland,Celtic_F.C.,Liverpool,England,Liverpool_F.C.,"Celtic Park, Glasgow",Semi-finals,76397,1965–66 European Cup Winners' Cup,1965-66
39,ICFC,1955-06-04,0-5,Basel XI,Switzerland,Basel,London XI,England,London_XI,"St. Jakob Stadium, Basel",Group stage,NaN,1955–58 Inter-Cities Fairs Cup,1955-58
0,UCL,1955-09-04,3-3,Sporting CP,Portugal,Sporting_CP,Partizan,Socialist Federal Republic of Yugoslavia,FK_Partizan,"Estádio Nacional, Lisbon",First round,30000,1955–56 European Cup,1955-56
1,UCL,1955-09-07,6-3,Vörös Lobogó,Hungary,MTK_Budapest_FC,Anderlecht,Belgium,R.S.C._Anderlecht,"Népstadion, Budapest",First round,35000,1955–56 European Cup,1955-56
2,UCL,1955-09-08,0-2,Servette,Switzerland,Servette_FC,Real Madrid,Spain,Real_Madrid_CF,"Charmilles Stadium, Geneva",First round,7000,1955–56 European Cup,1955-56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26951,EL,2024-05-22,3-0,Atalanta,Italy,Atalanta_BC,Bayer Leverkusen,Germany,Bayer_04_Leverkusen,"Aviva Stadium, Dublin",Final,4,2023–24 UEFA Europa League knockout phase,2023-24
26534,UECL,2024-05-29,1-0 (a.e.t.),Olympiacos,Greece,Olympiacos_F.C.,Fiorentina,Italy,ACF_Fiorentina,"Agia Sophia Stadium, Athens",Final,2,2023–24 UEFA Europa Conference League knockout...,2023-24
26304,UCL,2024-06-01,0-2,Borussia Dortmund,Germany,Borussia_Dortmund,Real Madrid,Spain,Real_Madrid_CF,"Wembley Stadium, London",Final,8,2023–24 UEFA Champions League knockout phase,2023-24
1029,UCL,NaT,Match unplayed,Derry City,Northern Ireland,Derry_City_F.C.,Anderlecht,Belgium,R.S.C._Anderlecht,NaN,First round,NaN,1965–66 European Cup,1965-66


In [174]:
df[df['competition'].isnull()]

,competition,date,score,homeTeam,homeTeamCountry,homeTeamUrl,awayTeam,awayTeamCountry,awayTeamUrl,location,phase,attendance,competitonOriginal,season
7035,None,1994-05-18,4-0,Milan,Italy,/wiki/AC_Milan,Barcelona,Spain,/wiki/FC_Barcelona,"Olympic Stadium, Athens",Knockout stage,70000,NaN,None


In [175]:
df.to_csv(os.path.join('data','matches.csv'), index=False)